**Modelo 4**


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as mat
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
train_df = pd.read_csv("/content/drive/MyDrive/GSCred/cs-training.csv", index_col=0)
test_df  = pd.read_csv("/content/drive/MyDrive/GSCred/cs-test.csv", index_col=0)
#submission = pd.read_csv('../input/GiveMeSomeCredit/sampleEntry.csv')

**Outliers**

Comprobación y eliminación de valores atípicos

La siguiente función filtra el DataFrame de acuerdo con la condición específica que solo se deben incluir las filas donde el valor en la columna 'RevolvingUtilizationOfUnsecuredLines' sea menor o igual a 10.

In [3]:
train_df = train_df[train_df['RevolvingUtilizationOfUnsecuredLines'] <= 10]

Comprobar y eliminar valores atípicos para NumberOfTimes90DaysLate, NumberOfTime30-59DaysPastDueNotWorse, NumberOfTime60-89DaysPastDueNotWorse

In [4]:
lower = 0
upper = 90

def remove_outliers(data,column):
    outliers_removed = data[data[column].between(lower, upper)]
    return outliers_removed

In [5]:
outlier_removed_train_df = remove_outliers(train_df,'NumberOfTimes90DaysLate')
outlier_removed_train_df = remove_outliers(outlier_removed_train_df,'NumberOfTime30-59DaysPastDueNotWorse')
outlier_removed_train_df = remove_outliers(outlier_removed_train_df,'NumberOfTime60-89DaysPastDueNotWorse')
train_df = outlier_removed_train_df

Manejo de valores faltantes

In [6]:
scaler = MinMaxScaler()
train_df_normalize = pd.DataFrame(scaler.fit_transform(train_df), columns = train_df.columns)
train_df_normalize.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1.0,0.086550,0.412844,0.153846,2.435759e-06,0.003031,0.224138,0.000000,0.111111,0.0,0.10
1,0.0,0.108130,0.366972,0.000000,3.696982e-07,0.000864,0.068966,0.000000,0.000000,0.0,0.05
2,0.0,0.074355,0.348624,0.076923,2.581822e-07,0.001011,0.034483,0.058824,0.000000,0.0,0.00
3,0.0,0.026414,0.275229,0.000000,1.093528e-07,0.001097,0.086207,0.000000,0.000000,0.0,0.00
4,0.0,0.102491,0.449541,0.076923,7.560939e-08,0.021134,0.120690,0.000000,0.018519,0.0,0.00


In [7]:
imputer = KNNImputer()
imputer.fit(train_df_normalize)
train_df_no_null_values = imputer.transform(train_df_normalize)

In [8]:
train_data = pd.DataFrame(train_df_no_null_values)
train_data.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
dtype: int64

In [9]:
x = train_df.drop("SeriousDlqin2yrs", axis=1)
y = train_df["SeriousDlqin2yrs"]
x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149490 entries, 1 to 150000
Data columns (total 10 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   RevolvingUtilizationOfUnsecuredLines  149490 non-null  float64
 1   age                                   149490 non-null  int64  
 2   NumberOfTime30-59DaysPastDueNotWorse  149490 non-null  int64  
 3   DebtRatio                             149490 non-null  float64
 4   MonthlyIncome                         119948 non-null  float64
 5   NumberOfOpenCreditLinesAndLoans       149490 non-null  int64  
 6   NumberOfTimes90DaysLate               149490 non-null  int64  
 7   NumberRealEstateLoansOrLines          149490 non-null  int64  
 8   NumberOfTime60-89DaysPastDueNotWorse  149490 non-null  int64  
 9   NumberOfDependents                    145610 non-null  float64
dtypes: float64(4), int64(6)
memory usage: 12.5 MB


División del conjunto de datos en conjunto de entrenamiento y prueba


In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

XGBClassifier Model

In [11]:
gs = XGBClassifier()

parameters = {'n_estimators':[1018],
              'colsample_bytree':[0.5],
              'subsample': [0.8],
              'max_depth': [6],
              'min_child_weight': [3],
              'learning_rate': [0.01],
              'reg_alpha' :[1],
              'reg_lambda':[1],
              'scale_pos_weight':[1]
             }


**GridSearchCV**

Es una herramienta de búsqueda de hiperparámetros (también conocida como hiperparametrización) en Python que se utiliza para encontrar los mejores valores de hiperparámetros para un modelo de aprendizaje automático.

Este hace una búsqueda exhaustiva a través de todas las combinaciones posibles de hiperparámetros especificados.

Mientras que **RandomizedSearchCV()** en lugar de evaluar todas las combinaciones posibles, hace una busqueda aleatoria de hiperparametros.

In [12]:
from sklearn.model_selection import RandomizedSearchCV

xgb_grid = RandomizedSearchCV(gs,
                         parameters,
                         scoring='roc_auc',
                         cv=10,
                         n_jobs=-1,
                         verbose=1)




In [13]:
lgbm_fit = xgb_grid.fit(x_train, y_train)
# Entrena el modelo xgb_grid con los datos de x_train y y_train. Una vez
# ejecutado el entrenamiento se podrá acceder a los mejores hiperparámetros
# utilizando el atributo best_params_ ie best_params = xgb_grid.best_params_
# También se puede acceder al mejor modelo entrenado utilizando el atributo
# best_estimator_: best_model = xgb_grid.best_estimator_
# De esta manera, se pueden encontrar los mejores hiperparámetros y el mejor
# modelo entrenado para un conjunto de datos específico.

Fitting 10 folds for each of 1 candidates, totalling 10 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [14]:
print(lgbm_fit.best_score_)
print(lgbm_fit.best_params_)

0.8651065009299742
{'subsample': 0.8, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 1, 'n_estimators': 1018, 'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.01, 'colsample_bytree': 0.5}


In [15]:
lgbm_fit.fit(x_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 10 folds for each of 1 candidates, totalling 10 fits


RandomizedSearchCV(cv=10,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rat...
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.5],
                                        'learning_rate': [0.01],
                                        'max_depth': [6],
                                        'min_child_weight': [3],
                                        'n_estimators': [1018],
                                        'reg_alpha': [1], 'reg_lambda': [1],
                                        'scale_pos_weight': [1],
                                        'subsample': [0.8]},
                   scoring='roc_auc', verbose=1)

In [16]:
print("training accuracy: {:.2f}".format(lgbm_fit.score(x_train, y_train) * 100))
print("validation accuracy: {:.2f}".format(lgbm_fit.score(x_test, y_test) * 100))

training accuracy: 88.85
validation accuracy: 86.32


In [18]:
x = test_df.drop(["SeriousDlqin2yrs"], axis=1)
y = lgbm_fit.predict_proba(x)[:, 1]
ids = x.index.values

In [21]:
ids = np.arange(1,len(y)+1)
submission = pd.DataFrame( {'Id': ids, 'Probability': y})
submission.to_csv("/kaggle/working/submission.csv", index=False)